In [5]:
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from collections import Counter

In [4]:
# extra imports
from pandas import read_csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
from statsmodels.genmod.generalized_linear_model import GLM
from pandas.plotting import scatter_matrix
from scipy.stats import boxcox

### CONTEXTO DEL PROYECTO

League of Legends (también conocido como LoL) es un videojuego multijugador en línea de estrategia en tiempo real y de género MOBA (Multiplayer Online Battle Arena), en el cual dos equipos de cinco jugadores compiten para destruir la base del equipo contrario en un mapa simétrico. Cada jugador controla a un personaje, llamado campeón, que tiene habilidades únicas y distintas para luchar contra los enemigos y para ayudar a su equipo a ganar la partida. Los jugadores ganan experiencia y oro al derrotar a los enemigos y a los monstruos del mapa, lo que les permite mejorar sus habilidades y comprar objetos para fortalecer a sus campeones. 

League of Legends es uno de los videojuegos más populares y competitivos del mundo, con millones de jugadores en todo el mundo, por lo que hemos creído interesante estudiar datos de los **10 primeros minutos** de 10,000 partidas del rango Diamante (uno de los rangos usados para clasificar a los jugadores según su nivel) para intentar predecir el desenlace de estas partidas y analizar que factores tienen más importancia en los 10 primeros minutos a la hora deque uno de los dos equipos gane la partida.

Los datos han sido extraídos de *Kaggle* (https://www.kaggle.com/datasets/bobbyscience/league-of-legends-diamond-ranked-games-10-min?resource=download), pero han sido distribuídos originalmente por *Riot Games* (empresa creadora de League of Legends).

### LECTURA DEL DATASET

In [22]:
# Load the data
dataset = read_csv("league_dataset.csv", header=0, delimiter=',')
dataset.head()

,gameId,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,4519157822,0,28,2,1,9,6,11,0,0,...,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,4523371949,0,12,1,0,5,5,5,0,0,...,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,4521474530,0,15,0,0,7,11,4,1,1,...,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,4524384067,0,43,1,0,4,5,5,1,0,...,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,4436033771,0,75,4,0,6,6,6,0,0,...,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4


### LIMPIADO DEL DATASET

Antes de empezar a trabajar con el dataset, vamos a eliminar dos variables que no tienen repercusión alguna en el desenlace de una partida o que son redundantes:

- *gameId* : esta variable actua como identificador de cada partida, algo que no tiene influencia sobre la propia partida.

- *redCSPerMin* : esta variable es completamente dependiente de *redTotalMinionsKilled*, pues su valor es el esta ultima variable dividido entre 10 (minutos de partidatranscurridos en el momento en el que se toman los datos).

- *blueCSPerMin* : mismo caso que con *redCSPerMin*.

- *redGoldPerMin* y *blueGoldPerMin*: exactamente la misma situación que con las dos variables anteriores, sus valores son los de *redTotalGold* y *blueTotalGold* divididos entre 10 (minutos de partida transcurridos en el momento en el que se toman los datos).

- *redGoldDiff* y *blueGoldDiff* : estas variables toman el mismo valor pues son la diferencia entre el oro total de un equipo y el del otro, por lo que resultan variables redundantes.

- *redEliteMonsters* y *blueEliteMonsters* : estas variables son la suma de *redDragons* y *redHeralds*, y *blueDragons* y *blueHeralds* respectivamente, por lo que también son variables redundantes.

- *redExperienceDiff* y *blueExperienceDiff* : tal y como pasa con las variables que representan la diferencia de oro, estas representan la diferencia de experiencia por lo que son directamente dependientes de *redTotalExperience* y *blueTotalExperience*.

- *redDeaths* y *blueDeaths* : variables directamente ligadas a *blueKills* y *redKills*, por lo que son redundantes.

- *redFirstBlood* : variable directamente ligada a *blueFirstBlood*, si esta última toma valor 1 el equipo azul ha hecho la primera sangre, si toma valor 0 el equipo rojo ha hecho la primera sangre.

In [23]:
del dataset['gameId']
del dataset['redCSPerMin']
del dataset['blueCSPerMin']
del dataset['redGoldPerMin']
del dataset['blueGoldPerMin']
del dataset['redGoldDiff']
del dataset['blueGoldDiff']
del dataset['redEliteMonsters']
del dataset['blueEliteMonsters']
del dataset['redExperienceDiff']
del dataset['blueExperienceDiff']
del dataset['redDeaths']
del dataset['blueDeaths']
del dataset['redFirstBlood']

In [24]:
dataset.head(10)

,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueAssists,blueDragons,blueHeralds,blueTowersDestroyed,blueTotalGold,...,redKills,redAssists,redDragons,redHeralds,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled
0,0,28,2,1,9,11,0,0,0,17210,...,6,8,0,0,0,16567,6.8,17047,197,55
1,0,12,1,0,5,5,0,0,0,14712,...,5,2,1,1,1,17620,6.8,17438,240,52
2,0,15,0,0,7,4,1,0,0,16113,...,11,14,0,0,0,17285,6.8,17254,203,28
3,0,43,1,0,4,5,0,1,0,15157,...,5,10,0,0,0,16478,7.0,17961,235,47
4,0,75,4,0,6,6,0,0,0,16400,...,6,7,1,0,0,17404,7.0,18313,225,67
5,1,18,0,0,5,6,1,0,0,15899,...,3,2,0,0,0,15201,7.0,18060,221,59
6,1,18,3,1,7,7,1,0,0,16874,...,6,9,0,0,0,14463,6.4,15404,164,35
7,0,16,2,0,5,3,0,0,0,15305,...,13,11,1,0,0,17920,6.6,16938,157,54
8,0,16,3,0,7,8,0,0,0,16401,...,7,5,1,1,0,18380,7.2,19298,240,53
9,1,13,1,1,4,5,1,0,0,15057,...,5,4,0,0,0,16605,6.8,18379,247,43


### EXPLICACIÓN DE LAS VARIABLES

Para los usuarios que no hayan jugado jamás al videojuego daremos una breve explicación de cada variable: 

- *blueWins* : esta variable indica si el equipo azul se ha alzado vencedor de la partida, es la variable que se quiere predecir (1 para victoria azul, 0 para victoria roja).

- *blueWardsPlaced* : indica el número de *wards* colocados por el equipo azul. Un *ward* aporta visión del mapa al equipo que lo coloca, por lo que puede suponer una ventaja táctica si se coloca apropiadamente.

- **blueWardsDestroyed* : indica el número de *wards* rojos destruidos por el equipo azul.

- *blueMinionsKilled* : indica el número de súbditos eliminados por parte del equipo azul durante los 15 primeros minutos de partida.

- *blueJungleMinionsKilled* : indica el número de monstruos de la jungla eliminados por parte del equipo azul durante los 15 primeros minutos de partida.

- *blueAvgLevel* : indica la media de nivel del equipo azul al final de los 15 minutos de partida. Cada equipo consta de 5 jugadores y se obtiene la media de estos.

- *blueChampKills* : indica la cantidad de eliminaciones del equipo azul conseguidas en los 15 primeros minutos, es decir, la cantidad de *kills* del equipo azul. Una *kill* consiste en eliminar a un jugador del equipo enemigo (este jugador reaparece tras un periodo de tiempo, por lo que no hay uj¡n límite de eliminaciones).

- *blueHeraldKills* : "REVISAR"

- *blueDragonKills* : variable categórica que indica el número de dragones eliminados por el equipo azul en los 15 primeros minutos (en este tiempo se puede matar como mucho 1 dragón).

También están estas mismas variables para el equipo rojo.
